# کرونا در خاورمیانه
> مروری بر مبتلایان کرونا ، مرگ و میر و موارد جدید براساس کشور در خاورمیانه.

- comments: true
- author:  نیلوفر هوشمند ، سولماز عباسپور
- categories: [ خاورمیانه, مرور,نقشه]
- image: images/covid-middle-east.png
- permalink: /covid-middle-east-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
!pip install jdatetime
!pip install persiantools
import numpy as np
import pandas as pd
from jinja2 import Template
import jdatetime as jd
import os
from persiantools import digits
from IPython.display import HTML

Could not build wheels for jdatetime, since package 'wheel' is not installed.


Could not build wheels for persiantools, since package 'wheel' is not installed.


In [3]:
#hide

# FETCH
dirpath = os.path.dirname(os.getcwd())
base_url = 'https://raw.githubusercontent.com/solmazabbaspour/iran-covid19/master/_notebooks/'
paths = {
    'mapping': base_url + 'middle_east.csv' ,
    'overview': dirpath + '/_templates/middle_east.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df = df[df['Country/Region'].isin(mapping['replace.country'])]   # Just use list above
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
    df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
    df_table.head(15)
df_table['Country/Region'] = df_table['Country/Region'].replace({'Iran': 'ایران'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Turkey': 'ترکیه'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Israel': 'اسراییل'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Saudi Arabia': 'عربستان سعودی'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'United Arab Emirates': 'امارات متحده عربی'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Qatar': 'قطر'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Egypt': 'مصر'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Iraq': 'عراق'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Bahrain': 'بحرین'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Lebanon': 'لبنان'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Kuwait': 'کویت'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Jordan': 'اردن'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Oman': 'عمان'})
df_table['Country/Region'] = df_table['Country/Region'].replace({'Syria': 'سوریه'})

In [6]:
#hide
# Iran Middle east
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
#s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
#s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
#s_mi = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
s_iran = df_table[df_table['Country/Region'].eq('ایران')][metrics].sum().add_prefix('Iran ')
s_tu = df_table[df_table['Country/Region'].eq('ترکیه')][metrics].sum().add_prefix('Turkey ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_iran, **s_tu}
summary

{'updated': Timestamp('2020-05-15 00:00:00'),
 'since': Timestamp('2020-05-10 00:00:00'),
 'Cases': 420245,
 'Deaths': 12629,
 'Cases (+)': 47064,
 'Deaths (+)': 730,
 'Iran Cases': 116635,
 'Iran Deaths': 6902,
 'Iran Cases (+)': 9032,
 'Iran Deaths (+)': 262,
 'Turkey Cases': 146457,
 'Turkey Deaths': 4055,
 'Turkey Cases (+)': 7800,
 'Turkey Deaths (+)': 269}

In [7]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Bahrain,0,0,0,0,0,0,0,0,0,0,...,214,265,245,330,167,295,295,285,382,385
Egypt,0,0,0,0,0,0,0,0,0,0,...,387,393,495,488,436,346,347,338,398,399
Iran,0,0,0,0,0,0,0,0,0,0,...,1680,1485,1556,1529,1383,1683,1481,1958,1808,2102
Iraq,0,0,0,0,0,0,0,0,0,0,...,49,63,60,76,88,51,95,119,111,50
Israel,0,0,0,0,0,0,0,0,0,0,...,21,71,55,18,23,29,23,19,31,10


In [8]:
#hide_input
template = Template(get_template(paths['overview']))
days = digits.en_to_fa(str((summary['updated'] - summary['since']).days))
g_date_updated = pd.to_datetime(summary['updated'])
jdate_updated = jd.date.fromgregorian(year=g_date_updated.year,month=g_date_updated.month,day=g_date_updated.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_updated = digits.en_to_fa(jdate_updated)
g_date_since = pd.to_datetime(summary['since'])
jdate_since = jd.date.fromgregorian(year=g_date_since.year,month=g_date_since.month,day=g_date_since.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_since = digits.en_to_fa(jdate_since)
html = template.render(
   D=summary, table=df_table,  # REMOVE .head(20) to see all values
   newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
   np=np, pd=pd, days=days, digits=digits, 
   jdate_updated=jdate_updated, jdate_since=jdate_since)

HTML(f'<div>{html}</div>')

کشور,تعداد مبتلایان,تعداد مرگ,درصد مرگ میر
,,,از (+جدبد) ۲۱ / ۰۲ / ۱۳۹۹
ترکیه,"۱۴۶,۴۵۷(+۷,۸۰۰)","۴,۰۵۵(+۲۶۹)",2.8%
ایران,"۱۱۶,۶۳۵(+۹,۰۳۲)","۶,۹۰۲(+۲۶۲)",5.9%
عربستان سعودی,"۴۹,۱۷۶(+۱۰,۱۲۸)",۲۹۲(+۴۶),0.6%
قطر,"۲۹,۴۲۵(+۶,۹۰۵)",۱۴(+۰),0.0%
امارات متحده عربی,"۲۱,۸۳۱(+۳,۶۳۳)",۲۱۰(+۱۲),1.0%
اسراییل,"۱۶,۵۸۹(+۱۱۲)",۲۶۶(+۱۴),1.6%
کویت,"۱۲,۸۶۰(+۴,۱۷۲)",۹۶(+۳۸),0.7%
مصر,"۱۱,۲۲۸(+۱,۸۲۸)",۵۹۲(+۶۷),5.3%
بحرین,"۶,۵۸۳(+۱,۶۴۲)",۱۲(+۴),0.2%




طراحی از [نیلوفر هوشمند](https://github.com/nilouhou/middle_east)

تحلیل از [سولماز عباسپور](http://www.solmazabbaspour.com/)[^۱]


[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).